<a href="https://colab.research.google.com/github/lalesafarzade/Recommendation_system_Project/blob/lale/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Movie").config("spark.executor.memory", "1G").config("spark.executor.cores","4").getOrCreate()

In [5]:
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [6]:
uploaded = files.upload()

Saving ratings.csv to ratings.csv


In [8]:
url = '/content/movies.csv'

movie = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

movie.cache()

DataFrame[movieId: int, title: string, genres: string]

In [9]:
movie=movie.drop('genres')

In [10]:
url = '/content/ratings.csv'

rating = spark.read.format("csv") \
       .option("header", "true") \
       .option("inferSchema","true")\
       .load(url) 

rating.cache() #for faster re-use

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [11]:
df = rating.join(movie,['movieId'],how='inner').distinct()
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)



In [13]:
df.show(5)

+-------+------+------+---------+--------------------+
|movieId|userId|rating|timestamp|               title|
+-------+------+------+---------+--------------------+
|     45|102689|   4.0|822873600|   To Die For (1995)|
|     48|109832|   4.0|822873600|   Pocahontas (1995)|
|     60| 17794|   2.0|823185207|Indian in the Cup...|
|     61| 17794|   4.0|823185214|Eye for an Eye (1...|
|      2| 17794|   3.0|823185225|      Jumanji (1995)|
+-------+------+------+---------+--------------------+
only showing top 5 rows



In [14]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+-------+------+------+---------+-----+
|movieId|userId|rating|timestamp|title|
+-------+------+------+---------+-----+
|      0|     0|     0|        0|    0|
+-------+------+------+---------+-----+



In [15]:
# Find count for empty, None, Null, Nan with string literals.
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.show()

+-------+------+------+---------+-----+
|movieId|userId|rating|timestamp|title|
+-------+------+------+---------+-----+
|      0|     0|     0|        0| 1168|
+-------+------+------+---------+-----+



In [22]:
df1=df.filter(((df.title != "None")|(df.title != "NULL")|(df.title != "")))

In [26]:
df2 = df1.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df1.columns])

df2.show()

In [28]:
(training, test) = df1.randomSplit([0.75, 0.25])

In [29]:
als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop", nonnegative = True, implicitPrefs = False)

In [30]:
param_grid = ParamGridBuilder().addGrid(als.rank, [10, 50, 75, 100]) .addGrid(als.maxIter, [5, 50, 75, 100]) \
 .addGrid(als.regParam, [.01, .05, .1, .15]).build()

# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
# Print length of evaluator
print ("Number of models by using param_grid: ", len(param_grid))

Number of models by using param_grid:  64


In [31]:
model = als.fit(training)

In [32]:
predictions = model.transform(test)

In [33]:
predictions.show(n = 10)

+-------+------+------+----------+--------------------+----------+
|movieId|userId|rating| timestamp|               title|prediction|
+-------+------+------+----------+--------------------+----------+
|  31956|     1|   3.5|1147877610|          5x2 (2004)|  3.362339|
|   8327|     1|   5.0|1147879375|        Dolls (2002)| 3.7703807|
|   1237|     1|   5.0|1147868839|Seventh Seal, The...| 3.9777155|
|   3949|     1|   5.0|1147868678|Requiem for a Dre...| 3.8322875|
|   7939|     1|   2.5|1147869183|Through a Glass D...| 3.8715022|
|   8360|     1|   4.0|1147868682|      Shrek 2 (2004)| 3.1913037|
|   8685|     1|   1.0|1147878023|Miracle of Marcel...| 2.9775615|
|   8873|     1|   3.0|1147869094|Motorcycle Diarie...| 3.7942362|
|  27266|     1|   4.5|1147879365|         2046 (2004)| 3.6225216|
|   2012|     1|   2.5|1147868068|Back to the Futur...| 2.7081363|
+-------+------+------+----------+--------------------+----------+
only showing top 10 rows



In [40]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

ConnectionRefusedError: ignored